<script async src="https://www.googletagmanager.com/gtag/js?id=UA-59152712-8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-59152712-8');
</script>

# `GiRaFFE_HO` C code library: A-to-B code

## Author: Patrick Nelson

<a id='intro'></a>

**Notebook Status:** <font color=orange><b> Self-Validated </b></font>

**Validation Notes:** This code produces the expected magnetic fields for Exact Wald, Aligned Rotator, and Alfv&eacute;n Wave initial data.

# This module presents the functionality of [GiRaFFE_HO_A2B.py](../edit/GiRaFFE_HO/GiRaFFE_HO_A2B.py) and builds on it by providing C code to incorporate the generated header files into `GiRaFFE`.

## Introduction: 
This writes and documents the C code that `GiRaFFE_HO` uses to compute the magnetic fields from the vector potential. This is a relatively straightforward calculation, but requires care to be taken in the ghost zones. 

We will need to compute $B^i$ everywhere in order to evolve $\tilde{S}_i$. However, $B^i = \epsilon^{ijk} \partial_j A_k$ requires derivatives of $A_i$, so getting $B^i$ in the ghostzones (and not just on the interior) will require some finesse. A chief aspect of this will require using lower-order finite differencing in the ghost zones. To that end, we will create header files for each finite differencing order $\leq 10$, as well as upwinded- and downwinded-derivatives at 2nd order. These will let us compute the derivative at the outermost gridpoints.


<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

This notebook is organized as follows

1. [Step 1](#headers): Creating the header files
    1. [Step 1.a](#bu_to_print): A useful function
    1. [Step 1.b](#interior): Handling the interior and inner ghostzones
    1. [Step 1.c](#outermost): Handling the outermost ghostzones
    1. [Step 1.d](#library): Output from the .py file for validation
1. [Step 2](#ccode): C code
    1. [Step 2.a](#a2b): Deciding which header file to use
    1. [Step 2.b](#driver): Looping over all ghostzones
1. [Step 3](#code_validation): Code Validation against original C code
1. [Step 4](#latex_pdf_output): Output this notebook to $\LaTeX$-formatted PDF file

In [1]:
# Step 0: Add NRPy's directory to the path
# https://stackoverflow.com/questions/16780014/import-file-from-parent-directory
import os,sys
nrpy_dir_path = os.path.join("..")
if nrpy_dir_path not in sys.path:
    sys.path.append(nrpy_dir_path)

import os
import cmdline_helper as cmd
outdir = "GiRaFFE_HO/GiRaFFE_Ccode_validation/A2B/"
cmd.mkdir(outdir)

<a id='headers'></a>

## Step 1: Creating the header files \[Back to [top](#toc)\]
$$\label{headers}$$

We start in the usual way - import the modules we need and set `DIM = 3`. We will also import the Levi-Civita symbol from `WeylScalars_Cartesian` and use it to set the Levi-Civita tensor $\epsilon^{ijk} = [ijk]/\sqrt{\gamma}$.

In [2]:
# Step 1: The A-to-B driver
from outputC import *
import grid as gri
import indexedexp as ixp
import NRPy_param_funcs as par
import finite_difference as fin

# Set spatial dimension (must be 3 for BSSN)
DIM = 3
par.set_parval_from_str("grid::DIM",DIM)

# Register the gridfunction gammadet. This determinant will be calculated separately
gammadet = gri.register_gridfunctions("AUXEVOL","gammadet")

# Import the Levi-Civita symbol and build the corresponding tensor.
# We already have a handy function to define the Levi-Civita symbol in WeylScalars
import WeylScal4NRPy.WeylScalars_Cartesian as weyl
LeviCivitaDDD = weyl.define_LeviCivitaSymbol_rank3()
LeviCivitaUUU = ixp.zerorank3()
for i in range(DIM):
    for j in range(DIM):
        for k in range(DIM):
            LCijk = LeviCivitaDDD[i][j][k]
            #LeviCivitaDDD[i][j][k] = LCijk * sp.sqrt(gho.gammadet)
            LeviCivitaUUU[i][j][k] = LCijk / sp.sqrt(gammadet)


<a id='bu_to_print'></a>

### Step 1.a: A useful function \[Back to [top](#toc)\]
$$\label{bu_to_print}$$

We will now create the structures we need: register the gridfunctions `AD` and `BU`, declare `AD_dD` for $A_{i,j}$ and zero `BU` for $B^i$. We will also define a quick function `set_BU_to_print()`; this will allow us to quickly rebuild the data structure that we will pass to `fin.FD_outputC()` each time we change `BU`.

Also, we'll build the standard form of `BU` that we will use: $$B^i = \epsilon^{ijk} A_{k,j}.$$

In [3]:
# Step 1.a: A useful function
AD = ixp.register_gridfunctions_for_single_rank1("EVOL","AD")
BU = ixp.register_gridfunctions_for_single_rank1("AUXEVOL","BU")
AD_dD = ixp.declarerank2("AD_dD","nosym")
BU = ixp.zerorank1() # BU is already registered as a gridfunction, but we need to zero its values and declare it in this scope.
# We can use this function to compactly reset to expressions to print at each FD order.
def set_BU_to_print():
    return [lhrh(lhs=gri.gfaccess("out_gfs","BU0"),rhs=BU[0]),\
            lhrh(lhs=gri.gfaccess("out_gfs","BU1"),rhs=BU[1]),\
            lhrh(lhs=gri.gfaccess("out_gfs","BU2"),rhs=BU[2])]            

for i in range(DIM):
    for j in range(DIM):
        for k in range(DIM):
            BU[i] += LeviCivitaUUU[i][j][k] * AD_dD[k][j]


<a id='interior'></a>

### Step 1.b: Handling the interior and inner ghostzones \[Back to [top](#toc)\]
$$\label{interior}$$

Now, we can output the header files that will be used to calculate $B^i$ on the interior of the grid and on the non-outermost ghost zones. For each FD order $\leq 10$, we will set the NRPy+ parameter `FD_CENTDERIVS_ORDER` to that order, then run `fin.FD_outputC()`, taking care to use our new function `set_BU_to_print()` to define *what* we are writing to the file.

In [4]:
# Step 1.b: Grid interior and inner ghostzones
par.set_parval_from_str("finite_difference::FD_CENTDERIVS_ORDER", 10)
fin.FD_outputC(os.path.join(outdir,"B_from_A_order10.h"),set_BU_to_print(),params="outCverbose=False")

par.set_parval_from_str("finite_difference::FD_CENTDERIVS_ORDER", 8)
fin.FD_outputC(os.path.join(outdir,"B_from_A_order8.h"),set_BU_to_print(),params="outCverbose=False")

par.set_parval_from_str("finite_difference::FD_CENTDERIVS_ORDER", 6)
fin.FD_outputC(os.path.join(outdir,"B_from_A_order6.h"),set_BU_to_print(),params="outCverbose=False")

par.set_parval_from_str("finite_difference::FD_CENTDERIVS_ORDER", 4)
fin.FD_outputC(os.path.join(outdir,"B_from_A_order4.h"),set_BU_to_print(),params="outCverbose=False")

par.set_parval_from_str("finite_difference::FD_CENTDERIVS_ORDER", 2)
fin.FD_outputC(os.path.join(outdir,"B_from_A_order2.h"),set_BU_to_print(),params="outCverbose=False")

Wrote to file "GiRaFFE_HO/GiRaFFE_Ccode_validation/A2B/B_from_A_order10.h"
Wrote to file "GiRaFFE_HO/GiRaFFE_Ccode_validation/A2B/B_from_A_order8.h"
Wrote to file "GiRaFFE_HO/GiRaFFE_Ccode_validation/A2B/B_from_A_order6.h"
Wrote to file "GiRaFFE_HO/GiRaFFE_Ccode_validation/A2B/B_from_A_order4.h"
Wrote to file "GiRaFFE_HO/GiRaFFE_Ccode_validation/A2B/B_from_A_order2.h"


<a id='outermost'></a>

### Step 1.c: Handling the outermost ghostzones \[Back to [top](#toc)\]
$$\label{outermost}$$

Finally, we will write the files that will handle the outermost ghost zones. Each of these ghostzones represents one face of a cube. Since there is no data outside the cube, we will use an up- or downwinded derivative so that we use only interior data. (For example, on the +x ghostzone, we will use a downwinded derivative in the x-direction.) This is what creates the difference between these loops and the one [above](#bu_to_print) - while we are building from the same equation as before, we define a derivative in one direction to be either up- or downwinded. So, we will need to create a total of six files: 

In [5]:
# Step 1.c: Handling the outermost ghostzones
# For the outermost points, we'll need a separate file for each face. 
# These will correspond to an upwinded and a downwinded file for each direction.
AD_ddnD = ixp.declarerank2("AD_ddnD","nosym")
for i in range(DIM):
    BU[i] = 0
    for j in range(DIM):
        for k in range(DIM):
            if j==0:
                BU[i] += LeviCivitaUUU[i][j][k] * AD_ddnD[k][j]
            else:
                BU[i] += LeviCivitaUUU[i][j][k] * AD_dD[k][j]

fin.FD_outputC(os.path.join(outdir,"B_from_A_order2_dirx0_dnwind.h"),set_BU_to_print(),params="outCverbose=False")

AD_dupD = ixp.declarerank2("AD_dupD","nosym")
for i in range(DIM):
    BU[i] = 0
    for j in range(DIM):
        for k in range(DIM):
            if j==0:
                BU[i] += LeviCivitaUUU[i][j][k] * AD_dupD[k][j]
            else:
                BU[i] += LeviCivitaUUU[i][j][k] * AD_dD[k][j]

fin.FD_outputC(os.path.join(outdir,"B_from_A_order2_dirx0_upwind.h"),set_BU_to_print(),params="outCverbose=False")

for i in range(DIM):
    BU[i] = 0
    for j in range(DIM):
        for k in range(DIM):
            if j==1:
                BU[i] += LeviCivitaUUU[i][j][k] * AD_ddnD[k][j]
            else:
                BU[i] += LeviCivitaUUU[i][j][k] * AD_dD[k][j]

fin.FD_outputC(os.path.join(outdir,"B_from_A_order2_dirx1_dnwind.h"),set_BU_to_print(),params="outCverbose=False")
for i in range(DIM):
    BU[i] = 0
    for j in range(DIM):
        for k in range(DIM):
            if j==1:
                BU[i] += LeviCivitaUUU[i][j][k] * AD_dupD[k][j]
            else:
                BU[i] += LeviCivitaUUU[i][j][k] * AD_dD[k][j]

fin.FD_outputC(os.path.join(outdir,"B_from_A_order2_dirx1_upwind.h"),set_BU_to_print(),params="outCverbose=False")

for i in range(DIM):
    BU[i] = 0
    for j in range(DIM):
        for k in range(DIM):
            if j==2:
                BU[i] += LeviCivitaUUU[i][j][k] * AD_ddnD[k][j]
            else:
                BU[i] += LeviCivitaUUU[i][j][k] * AD_dD[k][j]

fin.FD_outputC(os.path.join(outdir,"B_from_A_order2_dirx2_dnwind.h"),set_BU_to_print(),params="outCverbose=False")
for i in range(DIM):
    BU[i] = 0
    for j in range(DIM):
        for k in range(DIM):
            if j==2:
                BU[i] += LeviCivitaUUU[i][j][k] * AD_dupD[k][j]
            else:
                BU[i] += LeviCivitaUUU[i][j][k] * AD_dD[k][j]

fin.FD_outputC(os.path.join(outdir,"B_from_A_order2_dirx2_upwind.h"),set_BU_to_print(),params="outCverbose=False")

Wrote to file "GiRaFFE_HO/GiRaFFE_Ccode_validation/A2B/B_from_A_order2_dirx0_dnwind.h"
Wrote to file "GiRaFFE_HO/GiRaFFE_Ccode_validation/A2B/B_from_A_order2_dirx0_upwind.h"
Wrote to file "GiRaFFE_HO/GiRaFFE_Ccode_validation/A2B/B_from_A_order2_dirx1_dnwind.h"
Wrote to file "GiRaFFE_HO/GiRaFFE_Ccode_validation/A2B/B_from_A_order2_dirx1_upwind.h"
Wrote to file "GiRaFFE_HO/GiRaFFE_Ccode_validation/A2B/B_from_A_order2_dirx2_dnwind.h"
Wrote to file "GiRaFFE_HO/GiRaFFE_Ccode_validation/A2B/B_from_A_order2_dirx2_upwind.h"


<a id='library'></a>

### Step 1.d: Output from the .py file for validation \[Back to [top](#toc)\]
$$\label{library}$$

Now, let's use the python function `GiRaFFE_HO_A2B` to generate the copies that will actually be used by `GiRaFFE_standalone`. [Below](#code_validation), we'll validate the code we wrote here and make sure it's the same.

In [6]:
gri.glb_gridfcs_list = []
import GiRaFFE_HO.GiRaFFE_HO_A2B as A2B
A2B.GiRaFFE_HO_A2B("GiRaFFE_HO/GiRaFFE_Ccode_library/A2B")

Wrote to file "GiRaFFE_HO/GiRaFFE_Ccode_library/A2B/B_from_A_order10.h"
Wrote to file "GiRaFFE_HO/GiRaFFE_Ccode_library/A2B/B_from_A_order8.h"
Wrote to file "GiRaFFE_HO/GiRaFFE_Ccode_library/A2B/B_from_A_order6.h"
Wrote to file "GiRaFFE_HO/GiRaFFE_Ccode_library/A2B/B_from_A_order4.h"
Wrote to file "GiRaFFE_HO/GiRaFFE_Ccode_library/A2B/B_from_A_order2.h"
Wrote to file "GiRaFFE_HO/GiRaFFE_Ccode_library/A2B/B_from_A_order2_dirx0_dnwind.h"
Wrote to file "GiRaFFE_HO/GiRaFFE_Ccode_library/A2B/B_from_A_order2_dirx0_upwind.h"
Wrote to file "GiRaFFE_HO/GiRaFFE_Ccode_library/A2B/B_from_A_order2_dirx1_dnwind.h"
Wrote to file "GiRaFFE_HO/GiRaFFE_Ccode_library/A2B/B_from_A_order2_dirx1_upwind.h"
Wrote to file "GiRaFFE_HO/GiRaFFE_Ccode_library/A2B/B_from_A_order2_dirx2_dnwind.h"
Wrote to file "GiRaFFE_HO/GiRaFFE_Ccode_library/A2B/B_from_A_order2_dirx2_upwind.h"


<a id='ccode'></a>

## Step 2: C code \[Back to [top](#toc)\]
$$\label{ccode}$$

With the header files created to actually calculate $B^i$ from $A_k$, we will write the C code to direct how those are used. We will start by including needed header files. We will then define `REAL` as type `double` (so that we can easily change it everywhere if we want to use a different precision) and the `IDX` macros that we use to access specific points in the gridfunction arrays. 

In [7]:
%%writefile $outdir/driver_AtoB.c
#include <math.h>
#include <stdio.h>
#include <stdlib.h>

#ifndef REAL
#define REAL double
#include "NGHOSTS.h" // A NRPy+-generated file, which is set based on FD_CENTDERIVS_ORDER.
#include "../CurviBoundaryConditions/gridfunction_defines.h"
#define IDX4(g,i,j,k) \
( (i) + Nxx_plus_2NGHOSTS[0] * ( (j) + Nxx_plus_2NGHOSTS[1] * ( (k) + Nxx_plus_2NGHOSTS[2] * (g) ) ) )
#define IDX3(i,j,k) ( (i) + Nxx_plus_2NGHOSTS[0] * ( (j) + Nxx_plus_2NGHOSTS[1] * (k) ) )
// Assuming idx = IDX3(i,j,k). Much faster if idx can be reused over and over:
#define IDX4pt(g,idx)   ( (idx) + (Nxx_plus_2NGHOSTS[0]*Nxx_plus_2NGHOSTS[1]*Nxx_plus_2NGHOSTS[2]) * (g) )
#endif

// In the standalone, these are already defined by the boundary conditions module.
/*const int MAXFACE = -1;
*const int NUL     = +0;
*const int MINFACE = +1;
*/

Overwriting GiRaFFE_HO/GiRaFFE_Ccode_validation/A2B//driver_AtoB.c


<a id='a2b'></a>

### Step 2.a: Deciding which header file to use \[Back to [top](#toc)\]
$$\label{a2b}$$

The rest of this file will consist of two functions. The first, `AtoB`, will loop over the region specified by `i0min`, `i0max`, `i1min`, `i1max`, `i2min`, and `i2max`. At each point therein, it will calculate $B^i$ at the specified finite-differencing order. Note that if `ORDER = 0` is passed to this function, it will instead use second-order derivatives with an up- or downwinded derivative in one direction as appropriate, which is specified by the parameters `FACEX0`, `FACEX1`, and `FACEX2`.

In [8]:
%%writefile -a $outdir/driver_AtoB.c
// Declare boundary condition FACE_UPDATE function,
// which fills in the ghost zones with successively
// lower order finite differencing
void AtoB(const int ORDER, const int Nxx_plus_2NGHOSTS[3], const REAL *in_gfs, REAL *auxevol_gfs, const REAL dxx[3],
          const int i0min, const int i0max, 
          const int i1min, const int i1max, 
          const int i2min, const int i2max, 
          const int FACEX0, const int FACEX1, const int FACEX2) {
  
  const REAL invdx0 = 1.0 / dxx[0];
  const REAL invdx1 = 1.0 / dxx[1];
  const REAL invdx2 = 1.0 / dxx[2];

  if(ORDER==8) {
    //printf("Computing A to B with Order = 8...\n");
    for(int i2=i2min;i2<i2max;i2++) for(int i1=i1min;i1<i1max;i1++) for(int i0=i0min;i0<i0max;i0++) {
        #include "B_from_A_order8.h"
    }
  } else if(ORDER==6) {
    //printf("Computing A to B with Order = 6...\n");
    for(int i2=i2min;i2<i2max;i2++) for(int i1=i1min;i1<i1max;i1++) for(int i0=i0min;i0<i0max;i0++) {
        #include "B_from_A_order6.h"
    }
  } else if(ORDER==4) {
    //printf("Computing A to B with Order = 4...\n");
    for(int i2=i2min;i2<i2max;i2++) for(int i1=i1min;i1<i1max;i1++) for(int i0=i0min;i0<i0max;i0++) {
        #include "B_from_A_order4.h"
    }
  } else if(ORDER==2) {
    //printf("Computing A to B with Order = 2...\n");
    for(int i2=i2min;i2<i2max;i2++) for(int i1=i1min;i1<i1max;i1++) for(int i0=i0min;i0<i0max;i0++) {
        #include "B_from_A_order2.h"
    } 
  } else if(ORDER==0) {
    if(FACEX0==MAXFACE) {
    //printf("Computing A to B at x = max...\n");
        for(int i2=i2min;i2<i2max;i2++) for(int i1=i1min;i1<i1max;i1++) for(int i0=i0min;i0<i0max;i0++) {
            #include "B_from_A_order2_dirx0_dnwind.h"
        }
    } else if(FACEX0==MINFACE) {
    //printf("Computing A to B at x = min...\n");
        for(int i2=i2min;i2<i2max;i2++) for(int i1=i1min;i1<i1max;i1++) for(int i0=i0min;i0<i0max;i0++) {
            #include "B_from_A_order2_dirx0_upwind.h"
        }
    } else if(FACEX1==MAXFACE) {
    //printf("Computing A to B at y = max...\n");
        for(int i2=i2min;i2<i2max;i2++) for(int i1=i1min;i1<i1max;i1++) for(int i0=i0min;i0<i0max;i0++) {
            #include "B_from_A_order2_dirx1_dnwind.h"
        }
    } else if(FACEX1==MINFACE) {
    //printf("Computing A to B at y = min...\n");
        for(int i2=i2min;i2<i2max;i2++) for(int i1=i1min;i1<i1max;i1++) for(int i0=i0min;i0<i0max;i0++) {
            #include "B_from_A_order2_dirx1_upwind.h"
        }
    } else if(FACEX2==MAXFACE) {
    //printf("Computing A to B at z = max...\n");
        for(int i2=i2min;i2<i2max;i2++) for(int i1=i1min;i1<i1max;i1++) for(int i0=i0min;i0<i0max;i0++) {
            #include "B_from_A_order2_dirx2_dnwind.h"
        }
    } else if(FACEX2==MINFACE) {
    //printf("Computing A to B at z = min...\n");
        for(int i2=i2min;i2<i2max;i2++) for(int i1=i1min;i1<i1max;i1++) for(int i0=i0min;i0<i0max;i0++) {
            #include "B_from_A_order2_dirx2_upwind.h"
        }
    } else {
        printf("ERROR. FACEX parameters not set properly.\n");
        exit(1);
    }
  } else {
    printf("ERROR. ORDER = %d not supported!\n",ORDER);
    exit(1);
  }
}


Appending to GiRaFFE_HO/GiRaFFE_Ccode_validation/A2B//driver_AtoB.c


<a id='driver'></a>

### Step 2.b: Looping over all ghostzones \[Back to [top](#toc)\]
$$\label{driver}$$

This function is responsible for driving the logic needed to compute $B^i$ from $A_i$ whenever it is needed in the main code (typically done along with applying BCs after a timestep in the main code). We first calculate the order that we want on the interior of the grid from the parameter `NGHOSTS`. We set the boundaries `imin` and `imax` such that they define the interior of the grid. We also poison the values of $B^i$ by setting them to `NaN` everywhere: this will allow us to immediately tell if something is causing us to skip a point in the rest of the algorithm. 

With the preliminaries handled, we call `AtoB` at `ORDER` (which is being used for the rest of the derivatives throughout `GiRaFFE`). Note that `imin` and `imax` are passed such that they define the interior of the grid, and that the `FACEX` parameters are all `NUL` here. 

In [9]:
%%writefile -a $outdir/driver_AtoB.c
void driver_A_to_B(const int Nxx[3],const int Nxx_plus_2NGHOSTS[3], const REAL dxx[3],const REAL *in_gfs,REAL *auxevol_gfs) {
  
  int ORDER = NGHOSTS*2;
  for(int ii=0;ii<Nxx_plus_2NGHOSTS[2]*Nxx_plus_2NGHOSTS[1]*Nxx_plus_2NGHOSTS[0];ii++) {
      auxevol_gfs[IDX4pt(BU0GF,ii)] = 1.0 / 0.0;
      auxevol_gfs[IDX4pt(BU1GF,ii)] = 1.0 / 0.0;
      auxevol_gfs[IDX4pt(BU2GF,ii)] = 1.0 / 0.0;
  }

  //printf("Starting A to B driver with Order = %d...\n",ORDER);
  int imin[3] = { NGHOSTS, NGHOSTS, NGHOSTS };
  int imax[3] = { Nxx[0]+NGHOSTS, Nxx[1]+NGHOSTS, Nxx[2]+NGHOSTS };
  AtoB(ORDER, Nxx_plus_2NGHOSTS, in_gfs, auxevol_gfs,dxx, imin[0],imax[0],imin[1],imax[1],imin[2],imax[2], NUL,NUL,NUL);


Appending to GiRaFFE_HO/GiRaFFE_Ccode_validation/A2B//driver_AtoB.c


Now, we will handle the ghost zones. We first step down the order the next lower even number. Then, for each face (taking care to match the region specified by the `imin` and `imax` parameters with the appropriate `FACEX` values), we calculate $B^i$ in the innermost ghostzone. We also decrement `imin` and increment `imax`. This serves two purposes: the next time through the loop, `AtoB` will operate on the next-outer ghost zone, and the size of each face will increase, eventually allowing us to act on each point. 

We then repeat the process for each lower (even!) order until we reach the outermost ghost zone. 

In [10]:
%%writefile -a $outdir/driver_AtoB.c
  while(ORDER>0) {
      // After updating each face, adjust imin[] and imax[] 
      //   to reflect the newly-updated face extents.
      ORDER -= 2;
      AtoB(ORDER, Nxx_plus_2NGHOSTS, in_gfs, auxevol_gfs,dxx, imin[0]-1,imin[0], imin[1],imax[1], imin[2],imax[2], MINFACE,NUL,NUL); 
      if(ORDER!=0) imin[0]--;
      AtoB(ORDER, Nxx_plus_2NGHOSTS, in_gfs, auxevol_gfs,dxx, imax[0],imax[0]+1, imin[1],imax[1], imin[2],imax[2], MAXFACE,NUL,NUL); 
      if(ORDER!=0) imax[0]++;

      AtoB(ORDER, Nxx_plus_2NGHOSTS, in_gfs, auxevol_gfs,dxx, imin[0],imax[0], imin[1]-1,imin[1], imin[2],imax[2], NUL,MINFACE,NUL); 
      if(ORDER!=0) imin[1]--;
      AtoB(ORDER, Nxx_plus_2NGHOSTS, in_gfs, auxevol_gfs,dxx, imin[0],imax[0], imax[1],imax[1]+1, imin[2],imax[2], NUL,MAXFACE,NUL); 
      if(ORDER!=0) imax[1]++;

      AtoB(ORDER, Nxx_plus_2NGHOSTS, in_gfs, auxevol_gfs,dxx, imin[0],imax[0], imin[1],imax[1], imin[2]-1,imin[2], NUL,NUL,MINFACE); 
      if(ORDER!=0) imin[2]--;
      AtoB(ORDER, Nxx_plus_2NGHOSTS, in_gfs, auxevol_gfs,dxx, imin[0],imax[0], imin[1],imax[1], imax[2],imax[2]+1, NUL,NUL,MAXFACE); 
      if(ORDER!=0) imax[2]++;
    }
}

Appending to GiRaFFE_HO/GiRaFFE_Ccode_validation/A2B//driver_AtoB.c


<a id='code_validation'></a>

# Step 3: Code Validation against original C code \[Back to [top](#toc)\]
$$\label{code_validation}$$

To validate the code in this tutorial we check for agreement between the files

1. that were written in this tutorial and
1. those that are stored in `GiRaFFE_HO/GiRaFFE_Ccode_library` or generated by `GiRaFFE_HO_A2B.py`


In [11]:
import difflib
import sys

# Define the directory that we wish to validate against:
valdir = "GiRaFFE_HO/GiRaFFE_Ccode_library/A2B/"

print("Printing difference between original C code and this code...")
# Open the files to compare
files_to_check = ["B_from_A_order10.h","B_from_A_order8.h","B_from_A_order6.h","B_from_A_order4.h","B_from_A_order2.h",
                  "B_from_A_order2_dirx0_dnwind.h","B_from_A_order2_dirx0_upwind.h",
                  "B_from_A_order2_dirx1_dnwind.h","B_from_A_order2_dirx1_upwind.h",
                  "B_from_A_order2_dirx2_dnwind.h","B_from_A_order2_dirx2_upwind.h",
                  "driver_AtoB.c"]

for file in files_to_check:
    print("Checking file " + file)
    with open(os.path.join(valdir+file)) as file1, open(os.path.join(outdir+file)) as file2:
        # Read the lines of each file
        file1_lines = file1.readlines()
        file2_lines = file2.readlines()
        num_diffs = 0
        for line in difflib.unified_diff(file1_lines, file2_lines, fromfile=os.path.join(valdir+file), tofile=os.path.join(outdir+file)):
            sys.stdout.writelines(line)
            num_diffs = num_diffs + 1
        if num_diffs == 0:
            print("No difference. TEST PASSED!")
        else:
            print("ERROR: Disagreement found with .py file. See differences above.")

Printing difference between original C code and this code...
Checking file B_from_A_order10.h
No difference. TEST PASSED!
Checking file B_from_A_order8.h
No difference. TEST PASSED!
Checking file B_from_A_order6.h
No difference. TEST PASSED!
Checking file B_from_A_order4.h
No difference. TEST PASSED!
Checking file B_from_A_order2.h
No difference. TEST PASSED!
Checking file B_from_A_order2_dirx0_dnwind.h
No difference. TEST PASSED!
Checking file B_from_A_order2_dirx0_upwind.h
No difference. TEST PASSED!
Checking file B_from_A_order2_dirx1_dnwind.h
No difference. TEST PASSED!
Checking file B_from_A_order2_dirx1_upwind.h
No difference. TEST PASSED!
Checking file B_from_A_order2_dirx2_dnwind.h
No difference. TEST PASSED!
Checking file B_from_A_order2_dirx2_upwind.h
No difference. TEST PASSED!
Checking file driver_AtoB.c
No difference. TEST PASSED!


# A New Approach


What follows is code (or, for now, a draft thereof) that will compute the Magnetic field *everywhere* without the use of numerous header files to handle the edge and vertex cases. To do so, we will determine whether each necessary FD template will run off the grid as a first step, then compensate accordingly. Here, the function `driver_A_to_B()` consists of the typical logic we use for BCs to make sure we hit each point in the boundary. In the macro `compute_A2B()`, We loop over a  subset of points passed to the macro from the driver function. At each point, we consider the derivatives in each of the three directions that we will need to compute the curl; if we are on the edge of the grid, we will shift the stencil we use on point so that it no longer extends to outside our computational domain. Once we have all the derivatives we will need, we simply calculate the magnetic field as we otherwise would.

**Current things to address**
* Optimizing the decision tree
    * List most common cases first
    * Can this be done with switch statements?
* Is there a preference whether `compute_A2B` is a macro or function
* Validate new algorithm

In [1]:
%%writefile $outdir/driver_AtoB_new.c
#define  compute_A2B(i0min,i0max, i1min,i1max, i2min,i2max) \
  for(int i2=i2min;i2<i2max;i2++) for(int i1=i1min;i1<i1max;i1++) for(int i0=i0min;i0<i0max;i0++) { \
      int dx_Ay,dx_Az,dy_Ax,dy_Az,dz_Ax,dz_Ay; \
      if (i0==0) { \
          dx_Ay = invdx0*(-3.0/2.0*in_gfs[IDX4(AD1GF, i0,i1,i2)] + 2*in_gfs[IDX4(AD1GF, i0+1,i1,i2)] - 1.0/2.0*in_gfs[IDX4(AD1GF, i0+2,i1,i2)]); \
          dx_Az = invdx0*(-3.0/2.0*in_gfs[IDX4(AD2GF, i0,i1,i2)] + 2*in_gfs[IDX4(AD2GF, i0+1,i1,i2)] - 1.0/2.0*in_gfs[IDX4(AD2GF, i0+2,i1,i2)]); \
      } \
      else if (i0==Nxx_plus_2NGHOSTS0-1) { \
          dx_Ay = invdx0*((3.0/2.0)*in_gfs[IDX4(AD1GF, i0,i1,i2)] - 2*in_gfs[IDX4(AD1GF, i0-1,i1,i2)] + (1.0/2.0)*in_gfs[IDX4(AD1GF, i0-2,i1,i2)]); \
          dx_Az = invdx0*((3.0/2.0)*in_gfs[IDX4(AD2GF, i0,i1,i2)] - 2*in_gfs[IDX4(AD2GF, i0-1,i1,i2)] + (1.0/2.0)*in_gfs[IDX4(AD2GF, i0-2,i1,i2)]); \
      } \
      else { \
          dx_Ay = invdx0*(-1.0/2.0*in_gfs[IDX4(AD1GF, i0-1,i1,i2)] + (1.0/2.0)*in_gfs[IDX4(AD1GF, i0+1,i1,i2)]); \
          dx_Az = invdx0*(-1.0/2.0*in_gfs[IDX4(AD2GF, i0-1,i1,i2)] + (1.0/2.0)*in_gfs[IDX4(AD2GF, i0+1,i1,i2)]); \
      } \
      if (i1==0) { \
          dy_Ax = invdx1*(-3.0/2.0*in_gfs[IDX4(AD0GF, i0,i1,i2)] + 2*in_gfs[IDX4(AD0GF, i0,i1+1,i2)] - 1.0/2.0*in_gfs[IDX4(AD0GF, i0,i1+2,i2)]); \
          dy_Az = invdx1*(-3.0/2.0*in_gfs[IDX4(AD2GF, i0,i1,i2)] + 2*in_gfs[IDX4(AD2GF, i0,i1+1,i2)] - 1.0/2.0*in_gfs[IDX4(AD2GF, i0,i1+2,i2)]); \
      } \
      else if (i1==Nxx_plus_2NGHOSTS1-1) { \
          dy_Ax = invdx1*((3.0/2.0)*in_gfs[IDX4(AD0GF, i0,i1,i2)] - 2*in_gfs[IDX4(AD0GF, i0,i1-1,i2)] + (1.0/2.0)*in_gfs[IDX4(AD0GF, i0,i1-2,i2)]); \
          dy_Az = invdx1*((3.0/2.0)*in_gfs[IDX4(AD2GF, i0,i1,i2)] - 2*in_gfs[IDX4(AD2GF, i0,i1-1,i2)] + (1.0/2.0)*in_gfs[IDX4(AD2GF, i0,i1-2,i2)]); \
      } \
      else { \
          dy_Ax = invdx1*(-1.0/2.0*in_gfs[IDX4(AD0GF, i0,i1-1,i2)] + (1.0/2.0)*in_gfs[IDX4(AD1GF, i0,i1+1,i2)]); \
          dy_Az = invdx1*(-1.0/2.0*in_gfs[IDX4(AD2GF, i0,i1-1,i2)] + (1.0/2.0)*in_gfs[IDX4(AD2GF, i0,i1+1,i2)]); \
      } \
      if (i2==0) { \
          dz_Ax = invdx2*(-3.0/2.0*in_gfs[IDX4(AD0GF, i0,i1,i2)] + 2*in_gfs[IDX4(AD0GF, i0,i1,i2+1)] - 1.0/2.0*in_gfs[IDX4(AD0GF, i0,i1,i2+2)]); \
          dz_Ay = invdx2*(-3.0/2.0*in_gfs[IDX4(AD1GF, i0,i1,i2)] + 2*in_gfs[IDX4(AD1GF, i0,i1,i2+1)] - 1.0/2.0*in_gfs[IDX4(AD1GF, i0,i1,i2+2)]); \
      } \
      else if (i2==Nxx_plus_2NGHOSTS2-1) { \
          dz_Ax = invdx2*((3.0/2.0)*in_gfs[IDX4(AD0GF, i0,i1,i2)] - 2*in_gfs[IDX4(AD0GF, i0,i1,i2-1)] + (1.0/2.0)*in_gfs[IDX4(AD0GF, i0,i1,i2-2)]); \
          dz_Ay = invdx2*((3.0/2.0)*in_gfs[IDX4(AD1GF, i0,i1,i2)] - 2*in_gfs[IDX4(AD1GF, i0,i1,i2-1)] + (1.0/2.0)*in_gfs[IDX4(AD1GF, i0,i1,i2-2)]); \
      } \
      else { \
          dz_Ax = invdx2*(-1.0/2.0*in_gfs[IDX4(AD0GF, i0,i1,i2-1)] + (1.0/2.0)*in_gfs[IDX4(AD0GF, i0,i1,i2+1)]); \
          dz_Ay = invdx2*(-1.0/2.0*in_gfs[IDX4(AD1GF, i0,i1,i2-1)] + (1.0/2.0)*in_gfs[IDX4(AD1GF, i0,i1,i2+1)]); \
      } \
      const double gammadet = auxevol_gfs[IDX4(GAMMADETGF, i0,i1,i2)]; \
      out_gfs[IDX4(BU0GF, i0,i1,i2)] = (dy_Az-dz_Ay)/gammadet; \
      out_gfs[IDX4(BU1GF, i0,i1,i2)] = (dz_Ax-dx_Az)/gammadet; \
      out_gfs[IDX4(BU2GF, i0,i1,i2)] = (dx_Ay-dy_Ax)/gammadet; \
  }

void driver_A_to_B(const paramstruct *restrict params,REAL *restrict in_gfs,REAL *restrict out_gfs) {
#include "set_Cparameters.h"
    const REAL invdx0 = 1.0 / dxx0;
    const REAL invdx1 = 1.0 / dxx1;
    const REAL invdx2 = 1.0 / dxx2;
    int imin[3] = { NGHOSTS, NGHOSTS, NGHOSTS };
    int imax[3] = { Nxx_plus_2NGHOSTS0-NGHOSTS, Nxx_plus_2NGHOSTS1-NGHOSTS, Nxx_plus_2NGHOSTS2-NGHOSTS };
#pragma omp parallel for
    for(int i2=imin[2];i2<imax[2];i2++) for(int i1=imin[1];i1<imax[1];i1++) for(int i0=imin[0];i0<imax[0];i0++) {
        // Should probably generalize this later to allow for other orders.
#include "B_from_A_order2.h"
    }
    for(int which_gz = 0; which_gz < NGHOSTS; which_gz++) {
        // After updating each face, adjust imin[] and imax[] 
        //   to reflect the newly-updated face extents.
        compute_A2B(imin[0]-1,imin[0], imin[1],imax[1], imin[2],imax[2]); imin[0]--;
        compute_A2B(imax[0],imax[0]+1, imin[1],imax[1], imin[2],imax[2]); imax[0]++;

        compute_A2B(imin[0],imax[0], imin[1]-1,imin[1], imin[2],imax[2]); imin[1]--;
        compute_A2B(imin[0],imax[0], imax[1],imax[1]+1, imin[2],imax[2]); imax[1]++;

        compute_A2B(imin[0],imax[0], imin[1],imax[1], imin[2]-1,imin[2]); imin[2]--;
        compute_A2B(imin[0],imax[0], imin[1],imax[1], imax[2],imax[2]+1); imax[2]++;
    }
}

IndentationError: unexpected indent (<ipython-input-1-58e48bf32933>, line 2)

<a id='latex_pdf_output'></a>

# Step 4: Output this notebook to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

The following code cell converts this Jupyter notebook into a proper, clickable $\LaTeX$-formatted PDF file. After the cell is successfully run, the generated PDF may be found in the root NRPy+ tutorial directory, with filename
[Tutorial-GiRaFFE_HO_C_code_library-A2B](TTutorial-GiRaFFE_HO_C_code_library-A2B.pdf) (Note that clicking on this link may not work; you may need to open the PDF file through another means.)

In [12]:
!jupyter nbconvert --to latex --template latex_nrpy_style.tplx Tutorial-GiRaFFE_HO_C_code_library-A2B.ipynb
!pdflatex -interaction=batchmode Tutorial-GiRaFFE_HO_C_code_library-A2B.tex
!pdflatex -interaction=batchmode Tutorial-GiRaFFE_HO_C_code_library-A2B.tex
!pdflatex -interaction=batchmode Tutorial-GiRaFFE_HO_C_code_library-A2B.tex
!rm -f Tut*.out Tut*.aux Tut*.log

[NbConvertApp] Converting notebook Tutorial-GiRaFFE_HO_C_code_library-A2B.ipynb to latex
[NbConvertApp] Writing 83993 bytes to Tutorial-GiRaFFE_HO_C_code_library-A2B.tex
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
